In [14]:
import pandas as pd
import numpy as np
import PyPDF2
import tabula
from pathlib import Path
import logging
import pdfplumber
import os
import sys
from datetime import datetime
import warnings

ImportError: cannot import name 'is_cjk_uncommon' from 'charset_normalizer.utils' (c:\Users\Jordan\miniconda3\envs\env_data_processor\lib\site-packages\charset_normalizer\utils.py)

In [5]:
# Load environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
# Setup logging
log_path = Path(os.getenv('LOG_PATH', './logs'))
log_path.mkdir(exist_ok=True)

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_path / 'data_extraction.log'),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

print("=== Environmental Data Extractor ===")
print(f"Using Python: {sys.executable}")
print(f"Pandas version: {pd.__version__}")
print(f"Working directory: {os.getcwd()}")

=== Environmental Data Extractor ===
Using Python: c:\Users\Jordan\miniconda3\envs\env_data_processor\python.exe
Pandas version: 2.3.1
Working directory: c:\Users\Jordan\OneDrive - St Leonard's College\Desktop\Computing\EnviroAI\EnviroData\notebooks


In [9]:
class DataExtractor:
    def __init__(self, data_path=None):
        self.data_path = Path(data_path) if data_path else Path(os.getenv('RAW_DATA_PATH', './data/raw'))
        self.excel_path = self.data_path / 'excel'
        self.pdf_path = self.data_path / 'pdf'
        
        logger.info(f"Initialized DataExtractor with path: {self.data_path}")
        
    def list_files(self):
        """List all Excel and PDF files in the data directories"""
        excel_files = list(self.excel_path.glob('*.xlsx')) + list(self.excel_path.glob('*.xls'))
        pdf_files = list(self.pdf_path.glob('*.pdf'))
        
        print(f"\nFound {len(excel_files)} Excel files:")
        for file in excel_files:
            print(f"  - {file.name}")
            
        print(f"\nFound {len(pdf_files)} PDF files:")
        for file in pdf_files:
            print(f"  - {file.name}")
            
        return excel_files, pdf_files
        
    def extract_excel_data(self, file_path):
        "Extract data from Excel files"
        excel_data = {}
        try:
            logger.info(f"Extracting Excel data from: {file_path.name}")
            
            # Get file info
            file_size = file_path.stat().st_size / (1024*1024)  # Size in MB
            if file_size > float(os.getenv('MAX_FILE_SIZE_MB', 100)):
                logger.warning(f"File {file_path.name} is large ({file_size:.1f}MB)")
            
            # Handle multiple sheets
            excel_file = pd.ExcelFile(file_path)
            sheets_data = {}
            
            print(f"\nProcessing {file_path.name} with {len(excel_file.sheet_names)} sheets:")
            
            for sheet in excel_file.sheet_names:
                print(f"  - Reading sheet: {sheet}")
                try:
                    df = pd.read_excel(file_path, sheet_name=sheet)
                    sheets_data[sheet] = {
                        'data': df,
                        'shape': df.shape,
                        'columns': list(df.columns),
                        'source_file': file_path.name,
                        'sheet_name': sheet,
                        'extraction_date': datetime.now().isoformat()
                    }
                    logger.info(f"Sheet '{sheet}': {df.shape[0]} rows, {df.shape[1]} columns")
                except Exception as e:
                    logger.error(f"Error reading sheet '{sheet}': {e}")
                    continue
                    
            return sheets_data
        except Exception as e:
            logger.error(f"Error extracting Excel data from {file_path}: {e}")
            return None
    
    def extract_pdf_tables(self, file_path):
        """Extract tables from PDF files using tabula-py (conda installed)"""
        try:
            logger.info(f"Extracting PDF tables from: {file_path.name}")
            
            # Check if Java is available (required for tabula-py)
            try:
                import subprocess
                subprocess.run(['java', '-version'], capture_output=True, check=True)
            except (subprocess.CalledProcessError, FileNotFoundError):
                logger.warning("Java not found. Tabula-py requires Java for PDF table extraction.")
                return self.extract_pdf_tables_fallback(file_path)
            
            # Using tabula for table extraction
            tables = tabula.read_pdf(
                str(file_path), 
                pages='all', 
                multiple_tables=True,
                pandas_options={'header': 0}
            )
            
            processed_tables = []
            for i, table in enumerate(tables):
                if not table.empty:
                    processed_tables.append({
                        'data': table,
                        'shape': table.shape,
                        'columns': list(table.columns),
                        'source_file': file_path.name,
                        'table_index': i,
                        'extraction_date': datetime.now().isoformat()
                    })
            
            logger.info(f"Extracted {len(processed_tables)} tables from {file_path.name}")
            return processed_tables
            
        except Exception as e:
            logger.error(f"Error extracting PDF tables from {file_path}: {e}")
            return self.extract_pdf_tables_fallback(file_path)
    
    def extract_pdf_tables_fallback(self, file_path):
        """Fallback PDF table extraction using pdfplumber"""
        try:
            logger.info(f"Using pdfplumber fallback for: {file_path.name}")
            
            tables = []
            with pdfplumber.open(file_path) as pdf:
                for page_num, page in enumerate(pdf.pages):
                    page_tables = page.extract_tables()
                    for table_num, table in enumerate(page_tables):
                        if table and len(table) > 1:  # Must have header and at least one row
                            # Convert to DataFrame
                            df = pd.DataFrame(table[1:], columns=table[0])
                            tables.append({
                                'data': df,
                                'shape': df.shape,
                                'columns': list(df.columns),
                                'source_file': file_path.name,
                                'page_number': page_num + 1,
                                'table_index': table_num,
                                'extraction_date': datetime.now().isoformat()
                            })
            
            logger.info(f"Extracted {len(tables)} tables using pdfplumber from {file_path.name}")
            return tables
            
        except Exception as e:
            logger.error(f"Error with pdfplumber extraction from {file_path}: {e}")
            return []
    
    def extract_pdf_text(self, file_path):
        """Extract text from PDF files"""
        try:
            logger.info(f"Extracting text from: {file_path.name}")
            
            # Try PyPDF2 first
            try:
                with open(file_path, 'rb') as file:
                    reader = PyPDF2.PdfReader(file)
                    text = ""
                    for page_num, page in enumerate(reader.pages):
                        page_text = page.extract_text()
                        text += f"\n--- Page {page_num + 1} ---\n{page_text}"
                    
                return {
                    'text': text,
                    'pages': len(reader.pages),
                    'source_file': file_path.name,
                    'extraction_method': 'PyPDF2',
                    'extraction_date': datetime.now().isoformat()
                    }
            except Exception as pypdf_error:
                logger.warning(f"PyPDF2 failed, trying pdfplumber: {pypdf_error}")
                
                # Fallback to pdfplumber
                with pdfplumber.open(file_path) as pdf:
                    text = ""
                    for page_num, page in enumerate(pdf.pages):
                        page_text = page.extract_text()
                        if page_text:
                            text += f"\n--- Page {page_num + 1} ---\n{page_text}"
                    
                return {
                    'text': text,
                    'pages': len(pdf.pages),
                    'source_file': file_path.name,
                    'extraction_method': 'pdfplumber',
                    'extraction_date': datetime.now().isoformat()
                    }
                    
        except Exception as e:
            logger.error(f"Error extracting text from {file_path}: {e}")
            return None
        
extractor = DataExtractor()

2025-08-21 11:27:46,382 - __main__ - INFO - Initialized DataExtractor with path: data\raw


In [10]:
def extract_pdf_text(self, file_path):
        """Extract text from PDF files"""
        try:
            logger.info(f"Extracting text from: {file_path.name}")
            
            # Try PyPDF2 first
            try:
                with open(file_path, 'rb') as file:
                    reader = PyPDF2.PdfReader(file)
                    text = ""
                    for page_num, page in enumerate(reader.pages):
                        page_text = page.extract_text()
                        text += f"\n--- Page {page_num + 1} ---\n{page_text}"
                    
                return {
                    'text': text,
                    'pages': len(reader.pages),
                    'source_file': file_path.name,
                    'extraction_method': 'PyPDF2',
                    'extraction_date': datetime.now().isoformat()
                    }
            except Exception as pypdf_error:
                logger.warning(f"PyPDF2 failed, trying pdfplumber: {pypdf_error}")
                
                # Fallback to pdfplumber
                with pdfplumber.open(file_path) as pdf:
                    text = ""
                    for page_num, page in enumerate(pdf.pages):
                        page_text = page.extract_text()
                        if page_text:
                            text += f"\n--- Page {page_num + 1} ---\n{page_text}"
                    
                return {
                    'text': text,
                    'pages': len(pdf.pages),
                    'source_file': file_path.name,
                    'extraction_method': 'pdfplumber',
                    'extraction_date': datetime.now().isoformat()
                    }
                    
        except Exception as e:
            logger.error(f"Error extracting text from {file_path}: {e}")
            return None


In [11]:
# List available files
print("\n" + "="*50)
print("SCANNING FOR FILES")
print("="*50)
excel_files, pdf_files = extractor.list_files()


SCANNING FOR FILES

Found 2 Excel files:
  - ghg-conversion-factors-2023-condensed-set-update.xlsx
  - ghg-emission-factors-hub-2025.xlsx

Found 1 PDF files:
  - ZyPDF.pdf


In [7]:
if excel_files:
    print("EXTRACTING EXCEL DATA")
    
    for excel_file in excel_files:
        excel_data = extractor.extract_excel_data(excel_file)
        
        if excel_data:
            print(f"\nSuccessfully extracted data from {excel_file.name}")
            for sheet_name, sheet_info in excel_data.items():
                print(f"Sheet '{sheet_name}': {sheet_info['shape']}")
                print(f"Columns: {sheet_info['columns'][:5]}{'...' if len(sheet_info['columns']) > 5 else ''}")

EXTRACTING EXCEL DATA
2025-08-14 10:34:03,119 - __main__ - INFO - Extracting Excel data from: ghg-conversion-factors-2023-condensed-set-update.xlsx

Processing ghg-conversion-factors-2023-condensed-set-update.xlsx with 26 sheets:
  - Reading sheet: Introduction
2025-08-14 10:34:04,706 - __main__ - INFO - Sheet 'Introduction': 43 rows, 4 columns
  - Reading sheet: What's new
2025-08-14 10:34:05,002 - __main__ - INFO - Sheet 'What's new': 49 rows, 4 columns
  - Reading sheet: Index
2025-08-14 10:34:05,318 - __main__ - INFO - Sheet 'Index': 103 rows, 4 columns
  - Reading sheet: Fuels
2025-08-14 10:34:05,563 - __main__ - INFO - Sheet 'Fuels': 159 rows, 8 columns
  - Reading sheet: Bioenergy
2025-08-14 10:34:05,844 - __main__ - INFO - Sheet 'Bioenergy': 83 rows, 6 columns
  - Reading sheet: Refrigerant & other
2025-08-14 10:34:06,085 - __main__ - INFO - Sheet 'Refrigerant & other': 229 rows, 7 columns
  - Reading sheet: Passenger vehicles
2025-08-14 10:34:06,425 - __main__ - INFO - Sheet '

c:\Users\Jordan\miniconda3\envs\env_data_processor\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
c:\Users\Jordan\miniconda3\envs\env_data_processor\lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [12]:
if pdf_files:
    print(f"\n{'='*50}")
    print("EXTRACTING PDF DATA")
    print("="*50)
    
    first_pdf = pdf_files[0]
    pdf_tables = extractor.extract_pdf_tables(first_pdf)
    
    if pdf_tables:
        print(f"\nSuccessfully extracted {len(pdf_tables)} tables from {first_pdf.name}")
        for i, table_info in enumerate(pdf_tables):
            print(f"Table {i+1}: {table_info['shape']}")

print(f"\n{'='*50}")
print("EXTRACTION COMPLETE")
print("="*50)
print("Check the logs directory for detailed extraction logs.")
print("Next: Run the data cleaning notebook (02_data_cleaning.ipynb)")


EXTRACTING PDF DATA
2025-08-21 11:28:31,871 - __main__ - INFO - Extracting PDF tables from: ZyPDF.pdf
2025-08-21 11:28:36,801 - tabula.io - WARNING - Got stderr: Aug 21, 2025 11:28:34 AM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JPEG2000 image: Java Advanced Imaging (JAI) Image I/O Tools are not installed

2025-08-21 11:28:36,805 - __main__ - ERROR - Error extracting PDF tables from data\raw\pdf\ZyPDF.pdf: 'utf-8' codec can't decode byte 0x92 in position 750: invalid start byte
2025-08-21 11:28:36,808 - __main__ - INFO - Using pdfplumber fallback for: ZyPDF.pdf
2025-08-21 11:28:36,811 - __main__ - ERROR - Error with pdfplumber extraction from data\raw\pdf\ZyPDF.pdf: name 'pdfplumber' is not defined

EXTRACTION COMPLETE
Check the logs directory for detailed extraction logs.
Next: Run the data cleaning notebook (02_data_cleaning.ipynb)
